# Project 4

This project applies programming skills acquired during the course to re-analyze microscopy data using Cellpose, comparing its segmentation results with those from the Nessys algorithm.

The authors of the PLOS Biology paper, "Nessys: A new set of tools for the automated detection of nuclei within intact tissues and dense 3D cultures" (Blin et al., 2019) introduced a novel nuclear segmentation method. Their images were processed using Nessys, an extensible Java-based automated cell nuclei identification software. The images and analytical results are available in the Image Data Resource (IDR):
- <https://doi.org/10.1371/journal.pbio.3000388>
- <https://idr.openmicroscopy.org/webclient/?show=project-801>

The IDR team has also converted the dataset into OME-Zarr format. Unlike Nessys, Cellpose was not used in the original study. 

## Local TIFF files

The dataset is available in the IDR, but first, we are going to download the B4_C3.tif file from the IDR and save it locally. We will then use Cellpose to segment the nuclei in the image and compare the results with those obtained using Nessys.

### 1. Install dependencies from [PyPI](https://pypi.org/)

In [ ]:
# Install dependencies
!pip install -r requirements.txt

In [2]:
# Import libraries
import bioio
import cellpose
import cellpose.models
import dask
from ipywidgets import *
import matplotlib.pyplot as plt
import numpy

### 2. Download C3_B4.tif from IDR

Using the filepath, it is possible to download locally the image by running from a terminal:

```bash
wget https://ftp.ebi.ac.uk/pub/databases/IDR/idr0062-blin-nuclearsegmentation/20190429-ftp/Blastocysts/B4_C3.tif
```

or, alternatively, by using the Python `ftputil` library:

In [3]:
import ftputil
import os

files = [
    "pub/databases/IDR/idr0062-blin-nuclearsegmentation/20190429-ftp/Blastocysts/B4_C3.tif",
    "pub/databases/IDR/idr0062-blin-nuclearsegmentation/20190429-ftp/Blastocysts/B4_C3_Manual.tif",
]

with ftputil.FTPHost("ftp.ebi.ac.uk", "anonymous") as host:
    names = host.listdir(host.curdir)
    for name in names:
        if name == "pub":  # only check the IDR data
            for file in files:
                filepath = os.path.join("data", os.path.basename(file))
                if host.path.isfile(file):
                    host.download(file, filepath)

### 3. Try to open in BioIO

The `bioio` package requires `bioio-tifffile` plugin to be able to load the image as downloaded.

In [4]:
filepath = os.path.join("data", "B4_C3.tif")
labelspath = os.path.join("data", "B4_C3_Manual.tif")

In [ ]:
# Load the image
images = bioio.BioImage(filepath)

# Load the label
labels = bioio.BioImage(labelspath)

# (T, C, Z, Y, X)
print(images.dims)
print(images.shape)

In [ ]:
def update(z=0, c=0):
    fig = plt.figure(figsize=(10, 10))
    plt.imshow(images.data[0, c, z, :, :])
    plt.tight_layout()
    fig.canvas.flush_events()


interact(
    update,
    z=widgets.IntSlider(
        value=0,
        min=0,
        max=images.data.shape[2] - 1,
        step=1,
        description="Select T",
        continuous_update=False,
    ),
    c=widgets.IntSlider(
        value=0,
        min=0,
        max=images.data.shape[1] - 1,
        step=1,
        description="Select C",
        continuous_update=False,
    ),
)

### 4. Select two Z-planes around the middle of the stack

In [ ]:
# Define the Z-planes to select
Z = [89, 90, 91]

# Select two Z-planes (indices 89 and 90) and stack them along a new axis
selected_images = numpy.stack([images.data[0, :, z, :, :] for z in Z], axis=1)

print(f"BioIO shape: {images.data.shape}")
print(f"Selected images shape: {selected_images.shape}")


### 5. Process the selected planes using Cellpose API with the default cyto model

In [8]:
# Convert the NumPy image data to a Dask array
dask_img = dask.array.from_array(
    selected_images,
    chunks=(
        selected_images.shape[0],
        1,
        selected_images.shape[2],
        selected_images.shape[3],
    ),
)

# Load the Cellpose model: cyto
model = cellpose.models.Cellpose(model_type="cyto", gpu=True)

# Delayed function for segmentation
@dask.delayed
def segment(image):
    # im_cyto_pre = gaussian(im_cytoplasm, sigma=0.5)
    # im_cyto_pre = rescale_intensity(im_cyto_pre,out_range=(0,1))
    # im_nuclei_pre = gaussian(im_nuclei, sigma=1)
    # im_nuclei_pre = rescale_intensity(im_nuclei_pre,out_range=(0,1))
    masks, _, _, _ = model.eval(image, diameter=50, min_size=10, channels=[0, 1])
    return masks

In [ ]:
%%time
# Perform segmentation with Dask
results = dask.compute(*[segment(dask_img[:,z,:,:]) for z in range(dask_img.shape[1])])

### 6. Display the segmentation results

We will use matplotlib to display the segmentation results. The output will show the Cellpose segmentation alongside the Nessys ground truth labels for comparison.

In [ ]:
def update(z=0):
    fig = plt.figure(figsize=(10, 10))
    fig, ax = plt.subplots(1, 3, figsize=(10, 5))

    # Original Image
    ax[0].imshow(dask_img[0][z], cmap="Blues")
    ax[0].imshow(dask_img[1][z], cmap="Reds")
    ax[0].set_title(f"Original - {Z[z] + 1}")
    ax[0].axis("off")
    
    # Segmented Image (Overlay)
    ax[1].imshow(dask_img[0][z], cmap="gray")
    ax[1].imshow(results[z], alpha=0.5, cmap="jet")
    ax[1].set_title(f"Segmentation - {Z[z] + 1}")
    ax[1].axis("off")
    
    # Labels Image
    ax[2].imshow(labels.data[0, 0, Z[z], :, :], cmap="jet")
    ax[2].set_title(f"Nessys' labels - {Z[z] + 1}")
    ax[2].axis("off")

    plt.tight_layout()
    fig.canvas.flush_events()

interact(
    update,
    z=widgets.IntSlider(
        value=0,
        min=0,
        max=len(results) - 1,
        step=1,
        description="Select T",
        continuous_update=False,
    ),
)

## OME-Zarr files

In this step, we will use the OME-Zarr format to analyze [6001247.zarr](https://uk1s3.embassy.ebi.ac.uk/idr/zarr/v0.1/6001247.zarr/0/) file. We will load the image using Dask and Cellpose and compare the results with the original Nessys segmentation.

### 1. Load 6001247.zarr file instead of TIFF

In [11]:
# Image information
IMAGE_ID = 6001247
ENPOINT_URL = 'https://uk1s3.embassy.ebi.ac.uk/'

In [12]:
# Define a function to load binary data from S3
def load_binary_from_s3(name, resolution='0'):
    root = '%s/%s/' % (name, resolution)
    return dask.array.from_zarr(ENPOINT_URL + root)

In [13]:
# Define the name of the Zarr file
name = 'idr/zarr/v0.1/%s.zarr' % IMAGE_ID

# Load the binary data from S3
data = load_binary_from_s3(name, resolution='0')

In [ ]:
print(f"Dask array shape: {data.shape}")

### 2. Read labels from the ome.zarr

In [15]:
# Define the name of the labels Zarr file
name = 'idr/zarr/v0.1/%s.zarr/labels' % IMAGE_ID

# Load the binary data from S3
labels = load_binary_from_s3(name)

In [ ]:
print(f"Dask array shape: {labels.shape}")

### 3. Analyze the image using Cellpose

In [ ]:
%%time
# Perform segmentation with Dask
results = dask.compute(*[segment(data[0,:,z,:,:]) for z in range(data.shape[2])])

In [ ]:
def update(z=0):
    fig = plt.figure(figsize=(10, 10))
    fig, ax = plt.subplots(1, 3, figsize=(10, 5))

    # Original Image
    ax[0].imshow(data[0, 0, z, :, :], cmap="Blues")
    ax[0].imshow(data[0, 1, z, :, :], cmap="Reds")
    ax[0].set_title(f"Original (z={z + 1})")
    ax[0].axis("off")
    
    # Segmented Image (Overlay)
    ax[1].imshow(data[0, 0, z, :, :], cmap="gray")
    ax[1].imshow(r[z], alpha=0.5, cmap="jet")
    ax[1].set_title(f"Segmentation (z={z + 1})")
    ax[1].axis("off")
    
    # Labels Image
    ax[2].imshow(labels[0, 0, z, :, :], cmap="jet")
    ax[2].set_title(f"Nessys' labels (z={z + 1})")
    ax[2].axis("off")

    plt.tight_layout()
    fig.canvas.flush_events()

interact(
    update,
    z=widgets.IntSlider(
        value=0,
        min=0,
        max=len(results) - 1,
        step=1,
        description="Select T",
        continuous_update=False,
    ),
)

### 4. Compare the labels with the Cellpose segmentation

We will compare Cellpose segmentations with Nessys labels using common segmentation metrics:
- IoU (Intersection over Union)
- Dice Coefficient (F1 Score for Image Segmentation)

In [19]:
# from skimage.metrics import adapted_rand_error
# from sklearn.metrics import jaccard_score
# import numpy as np

# def compute_metrics(pred_mask, gt_mask):
#     # Flatten the masks for comparison
#     pred_flat = pred_mask.ravel()
#     gt_flat = gt_mask.ravel()
    
#     # Compute Jaccard Index (IoU)
#     iou = jaccard_score(gt_flat, pred_flat, average='binary')

#     # Compute Dice Score
#     dice = 2 * np.sum(pred_flat * gt_flat) / (np.sum(pred_flat) + np.sum(gt_flat))

#     return iou, dice

# # Compute metrics for each plane
# metrics = [compute_metrics(masks[i], selected_nessys_labels[i]) for i in range(2)]

# # Display results
# for i, (iou, dice) in enumerate(metrics):
#     print(f"Z-plane {89 + i}: IoU = {iou:.4f}, Dice Score = {dice:.4f}")